<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/reranker_combined_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download JSON files with evaluation data

### Combine downloaded JSON files into one

In [1]:
import pandas as pd

data = pd.read_json('https://raw.githubusercontent.com/winterForestStump/thesis/main/retrieval/evaluation/eval_reranked.json')

In [2]:
new_values = {
    27: {'score': 'yes'},
    32: {'score': 'yes'},
    84: {'score': 'no'},
    91: {'score': 'yes'},
    104: {'score': 'yes'},
    130: {'score': 'no'},
    151: {'score': 'yes'},
    160: {'score': 'yes'},
    183: {'score': 'yes'},
    209: {'score': 'yes'},
    215: {'score': 'no'},
    252: {'score': 'yes'},
    254: {'score': 'yes'},
    256: {'score': 'yes'},
    278: {'score': 'no'},
    326: {'score': 'yes'}}

In [3]:
valid_answers = [{'score': 'yes'}, {'score': 'no'}]
data[~data['answer'].apply(lambda x: x in valid_answers)]

,question,answer,document
27,What are the company's pension obligations and...,{'score': 1},"[Effective January 1, 2016, for benefit plans ..."
32,What are the company's major operating expense...,[{'score': 'yes'}],[The decrease in International operating loss ...
84,What are the company's major operating expense...,[[]],"[In Millions\nNet sales:\nCPW\n$\n1,706.5\n$\n..."
91,How much is spent on research and development ...,None,[RESEARCH AND DEVELOPMENT\nOur research and de...
104,How much has the company invested in capital e...,[{'score': 'yes'}],[Net Cash Used in Investing Activities\nFiscal...
130,"What is the company's gross profit margin, and...",[{'score': 'no'}],[Assumption Amount\nDiscount rates used in the...
151,What is the company's operating income and how...,[{'score': 'yes'}],"[6,064\n7,024\n(13.7\n)%\n(a)\nAdjusted EBITDA..."
160,What is the effective tax rate for the company?,[{'score': 1}],"[Net\nTrademarks\n$\n2,443\n$\n(469\n)\n$\n1,9..."
183,What potential impact could legal issues have ...,None,[Other Matters\nIn the normal course of busine...
209,What is the effective tax rate for the company?,2.1,[NOTE 14 - INCOME TAXES\nThe domestic and fore...


In [4]:
new_values = {
    27: {'score': 'yes'},
    32: {'score': 'yes'},
    84: {'score': 'no'},
    91: {'score': 'yes'},
    104: {'score': 'yes'},
    130: {'score': 'no'},
    151: {'score': 'yes'},
    160: {'score': 'yes'},
    183: {'score': 'yes'},
    209: {'score': 'yes'},
    215: {'score': 'no'},
    252: {'score': 'yes'},
    254: {'score': 'yes'},
    256: {'score': 'yes'},
    278: {'score': 'no'},
    326: {'score': 'yes'}}

In [5]:
for index, new_value in new_values.items():
    data.at[index, 'answer'] = new_value

### Filter Approach 1 instances

In [6]:
score = 0
for i in range(len(data)):
  if (data['answer'][i] is not None) and (data['answer'][i]['score'] == 'yes'):
    score += 1
total_score = score/len(data)
total_score

0.7305389221556886

### Group the number of answers by questions

In [7]:
data_total = data.groupby('question')['answer'].count().reset_index()
data_total.rename(columns={'answer':'total'}, inplace=True)

### Filter instances with the 'yes' score

In [8]:
filtered_df = data[data['answer'].apply(lambda x: isinstance(x, dict) and x.get('score') == 'yes')]
len(filtered_df)

244

### Group the number of correct answers by questions

In [9]:
data_filtered = filtered_df.groupby('question')['answer'].count().reset_index()
data_filtered.rename(columns={'answer':'correct'}, inplace=True)

### Merge two tables: all and correct answers grouped by questions

In [10]:
new_data = data_total.merge(data_filtered, how='left', on='question')
new_data['correct'] = new_data['correct'].fillna(0)
len(new_data)

35

In [11]:
new_data['percentage,%'] = new_data['correct'] / new_data['total'] * 100
new_data.sort_values(by='percentage,%', inplace=True, ascending=False)
new_data['correct'] = new_data['correct'].astype(int)
new_data['percentage,%'] = new_data['percentage,%'].astype(int)

In [12]:
new_data.style

,question,total,correct,"percentage,%"
0,Are there any ongoing legal proceedings against the company?,10,10,100
1,Are there any tax-related risks or benefits for the company mentioned?,10,10,100
20,What is the company's cash flow generated from operations and are there any notable trends or fluctuations?,8,8,100
21,What is the company's cost of goods sold (COGS) and how does the COGS compare to the total revenue?,2,2,100
25,What is the company's geographic breakdown of revenue and are there any notable trends or shifts?,4,4,100
11,What are the company's critical accounting policies disclosed and how might changes in these policies affect financial statements?,13,13,100
29,What is the company's operating income and how does it compare to the previous years?,7,7,100
31,What is the effective tax rate for the company?,8,8,100
32,What is the total revenue generated by the company and how has the revenue changed over the past few years?,10,10,100
33,What potential impact could legal issues have on the business of the company?,13,13,100


In [14]:
print(new_data['total'].sum())
print(new_data['correct'].sum())
print(new_data['correct'].sum() / new_data['total'].sum() * 100)

334
244
73.05389221556887


### LateX syntax

In [13]:
latex_table = new_data.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrrr}
\toprule
question & total & correct & percentage,% \\
\midrule
Are there any ongoing legal proceedings against the company? & 10 & 10 & 100 \\
Are there any tax-related risks or benefits for the company mentioned? & 10 & 10 & 100 \\
What is the company's cash flow generated from operations and are there any notable trends or fluctuations? & 8 & 8 & 100 \\
What is the company's cost of goods sold (COGS) and how does the COGS compare to the total revenue? & 2 & 2 & 100 \\
What is the company's geographic breakdown of revenue and are there any notable trends or shifts? & 4 & 4 & 100 \\
What are the company's critical accounting policies disclosed and how might changes in these policies affect financial statements? & 13 & 13 & 100 \\
What is the company's operating income and how does it compare to the previous years? & 7 & 7 & 100 \\
What is the effective tax rate for the company? & 8 & 8 & 100 \\
What is the total revenue generated by the company and how has the rev